<a href="https://colab.research.google.com/github/anananacr/ml-hw/blob/main/homework_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [18]:
# @title
#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
#!unzip data.zip

In [19]:
data_shape = (3, 200, 200)

In [20]:
import torch.nn as nn
import torchvision.models as models

class HairClassifierMobileNet(nn.Module):
    def __init__(self, num_classes=1):
        super(HairClassifierMobileNet, self).__init__()

        # Load pre-trained MobileNetV2
        self.base_model = models.mobilenet_v2(weights='IMAGENET1K_V1')

        # Freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Remove original classifier
        self.base_model.classifier = nn.Identity()

        # Add custom layers
        self.conv2d = nn.Conv2d(out_channels = 32, kernel_size = (3,3), in_channels = 1280)
        self.relu = nn.ReLU()
        self.reduce_map_size = nn.MaxPool2d(kernel_size = (2, 2))
        self.linear_layer = nn.Linear(in_features=128, out_features=64)
        self.output_layer = nn.Linear(in_features=64, out_features=num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.reduce_map_size(x)
        x = torch.flatten(x,1)
        x = self.linear_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        return x

In [21]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()

def make_model(learning_rate=0.001):
    model = HairClassifierMobileNet(
        num_classes=1
    )

    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
    return model, optimizer

In [22]:
from torchsummary import summary
model, optimizer = make_model()
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 100, 100]             864
       BatchNorm2d-2         [-1, 32, 100, 100]              64
             ReLU6-3         [-1, 32, 100, 100]               0
            Conv2d-4         [-1, 32, 100, 100]             288
       BatchNorm2d-5         [-1, 32, 100, 100]              64
             ReLU6-6         [-1, 32, 100, 100]               0
            Conv2d-7         [-1, 16, 100, 100]             512
       BatchNorm2d-8         [-1, 16, 100, 100]              32
  InvertedResidual-9         [-1, 16, 100, 100]               0
           Conv2d-10         [-1, 96, 100, 100]           1,536
      BatchNorm2d-11         [-1, 96, 100, 100]             192
            ReLU6-12         [-1, 96, 100, 100]               0
           Conv2d-13           [-1, 96, 50, 50]             864
      BatchNorm2d-14           [-1, 96,

In [23]:
import os
from torch.utils.data import Dataset
from PIL import Image

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [24]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [25]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.4655, Acc: 0.8215, Val Loss: 0.1594, Val Acc: 0.9801
Epoch 2/10, Loss: 0.1145, Acc: 0.9663, Val Loss: 0.0966, Val Acc: 0.9701
Epoch 3/10, Loss: 0.1037, Acc: 0.9588, Val Loss: 0.2091, Val Acc: 0.9154
Epoch 4/10, Loss: 0.0804, Acc: 0.9738, Val Loss: 0.0423, Val Acc: 0.9851
Epoch 5/10, Loss: 0.0533, Acc: 0.9863, Val Loss: 0.0410, Val Acc: 0.9851
Epoch 6/10, Loss: 0.0339, Acc: 0.9913, Val Loss: 0.0527, Val Acc: 0.9801
Epoch 7/10, Loss: 0.0369, Acc: 0.9813, Val Loss: 0.2058, Val Acc: 0.9055
Epoch 8/10, Loss: 0.2106, Acc: 0.9451, Val Loss: 0.1255, Val Acc: 0.9453
Epoch 9/10, Loss: 0.0730, Acc: 0.9800, Val Loss: 0.0747, Val Acc: 0.9701
Epoch 10/10, Loss: 0.0445, Acc: 0.9888, Val Loss: 0.0660, Val Acc: 0.9751


In [29]:
np.median(history['acc'])

np.float64(0.9769038701622972)

In [30]:
np.std(history['loss'])

np.float64(0.12493360295541506)

In [33]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ), # ImageNet normalization
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
])


test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [34]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.1428, Acc: 0.9401, Val Loss: 0.0918, Val Acc: 0.9602
Epoch 2/10, Loss: 0.0774, Acc: 0.9800, Val Loss: 0.0951, Val Acc: 0.9602
Epoch 3/10, Loss: 0.0840, Acc: 0.9663, Val Loss: 0.0379, Val Acc: 0.9900
Epoch 4/10, Loss: 0.0663, Acc: 0.9725, Val Loss: 0.1048, Val Acc: 0.9652
Epoch 5/10, Loss: 0.0813, Acc: 0.9663, Val Loss: 0.0591, Val Acc: 0.9701
Epoch 6/10, Loss: 0.1416, Acc: 0.9513, Val Loss: 0.0571, Val Acc: 0.9801
Epoch 7/10, Loss: 0.0440, Acc: 0.9825, Val Loss: 0.0522, Val Acc: 0.9751
Epoch 8/10, Loss: 0.0521, Acc: 0.9838, Val Loss: 0.0351, Val Acc: 0.9801
Epoch 9/10, Loss: 0.0546, Acc: 0.9800, Val Loss: 0.0394, Val Acc: 0.9801
Epoch 10/10, Loss: 0.0567, Acc: 0.9750, Val Loss: 0.0419, Val Acc: 0.9851


In [36]:
np.mean(history['val_loss'])

np.float64(0.06144786900144959)

In [41]:
np.mean(history['val_acc'][5:])

np.float64(0.9800995024875622)